<a href="https://colab.research.google.com/github/haitongw/MIE1517_Project/blob/main/distracted_driver_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F